In [1]:
# Import libraries
import mediapipe as mp
import cv2

#from mediapipe.tasks import python
#from mediapipe.tasks.python import vision
# from google.colab.patches import cv2_imshow
# import numpy as np
# import matplotlib.pyplot as plt

In [ ]:
# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Initialize video capture
cap = cv2.VideoCapture(0)

# Indices for the nose tip and chin landmarks
NOSE_TIP_INDEX = 1
CHIN_INDEX = 152
RIGHT_INDEX = 234
LEFT_INDEX = 454

# Set drawing specifications
nose_chin_specs = mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)

# Create Face Mesh object with specific settings
with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
) as face_mesh:

    # Check if the video source is available
    if not cap.isOpened():
        print("Error: Could not open video source:(")
    else:
        # Process each video frame
        while cap.isOpened():
            success, image = cap.read()

            if not success:
                print("Ignoring empty camera frame:(")
                continue

            # Convert the image to RGB for face mesh processing
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(image_rgb)

            # If face landmarks are detected
            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    # Extract the nose tip and chin coordinates
                    nose_tip = face_landmarks.landmark[NOSE_TIP_INDEX]
                    chin = face_landmarks.landmark[CHIN_INDEX]
                    right = face_landmarks.landmark[RIGHT_INDEX]
                    left = face_landmarks.landmark[LEFT_INDEX]

                    # Convert normalized coordinates to pixel values
                    h, w, _ = image.shape
                    nose_tip_coords = (int(nose_tip.x * w), int(nose_tip.y * h))
                    chin_coords = (int(chin.x * w), int(chin.y * h))
                    right_coords = (int(right.x * w), int(right.y * h))
                    left_coords = (int(left.x * w), int(left.y * h))

                    # Draw the nose tip and chin points on the image
                    cv2.circle(image, nose_tip_coords, 5, (0, 255, 0), -1)
                    cv2.circle(image, chin_coords, 5, (0, 255, 0), -1)
                    cv2.circle(image, right_coords, 5, (0, 255, 0), -1)
                    cv2.circle(image, left_coords, 5, (0, 255, 0), -1)

                    # Optionally, draw a line between the nose tip and chin
                    cv2.line(image, nose_tip_coords, chin_coords, (255, 0, 0), 2)
                    cv2.line(image, right_coords, left_coords, (255, 0, 0), 2)

            # Display the output
            cv2.imshow("Nose Tip and Chin Detection", cv2.flip(image, 1))

            # Break the loop if 'q' is pressed
            if cv2.waitKey(100) == ord('q'):
                break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)